In [ ]:
import pandas as pd
df = pd.read_csv('queryset_CNN.csv')
print(df.shape)
print(df.dtypes)

(32, 3)
doc_id         int64
author_id      int64
authorList    object
dtype: object


In [ ]:

for index, row in df.iterrows():
    doc_id = row.doc_id

    author_id = row.author_id

    import ast
    authorList = ast.literal_eval(row.authorList)
    
    candidate = len(authorList)

    test = "batch10" # change before run

    level = "word"

    iterations = 30

    dropout = 0.5

    samples = 3200

    dimensions = 200

    loc = authorList.index(author_id)

    printstate = (("doc_id = %s, candidate = %s, ") % (str(doc_id), str(candidate)))
    printstate += (("dimensions = %s, samples = %s, ") % (str(dimensions), str(samples)))
    printstate += (("\niterations = %s, dropout = %s, test = %s") % (str(iterations), str(dropout), str(test)))

    print("Current test: %s" % (str(printstate)))

    
    import UpdateDB as db

    case = db.checkOldCNNDiff(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                              samples = samples, iterations = iterations, dropout = dropout,
                              test = test)

    if case == False:

        print("Running: %12s" % (str(printstate)))

        import StyloNeural as Stylo
        (labels_index, history, train_acc, val_acc, samples) = Stylo.getResults(
            doc_id = doc_id, authorList = authorList[:], 
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )

        (predY, testY) = Stylo.getTestResults(
            doc_id = doc_id, authorList = authorList[:], labels_index = labels_index,
            level = level, glove = '../../glove/', dimensions = dimensions, 
            samples = samples, nb_epoch = iterations, dropout = dropout, batch_size = 10 )

        loc = testY

        test_acc = predY[loc]

        test_bin = 0

        if(predY.tolist().index(max(predY)) == testY):
            test_bin = 1
        
        import UpdateDB as db
        case = db.updateresultOldCNNDiff(doc_id = doc_id, candidate = candidate, dimensions = dimensions,
                                         samples = samples, iterations = iterations, dropout = dropout, 
                                         train_acc = train_acc, val_acc = val_acc,
                                         test_acc = test_acc, test_bin = test_bin,
                                         test = test)
                                     
        del Stylo

        from keras import backend as K
        K.clear_session()

        import time
        time.sleep(10)
        
        from IPython.display import clear_output

        clear_output()

    else:
        print("Skipped: %12s" % (str(printstate)))


# import pandas as pd
# df = pd.DataFrame(output)
# df.to_csv("styloout.csv", index = False, encoding='utf-8')

import time
time.sleep(10)

Current test: doc_id = 168, candidate = 3, dimensions = 200, samples = 3200, 
iterations = 30, dropout = 0.5, test = batch10
Execution completed
Running: doc_id = 168, candidate = 3, dimensions = 200, samples = 3200, 
iterations = 30, dropout = 0.5, test = batch10
Level = Word
File used: glove.6B.200d.txt
Found 400000 word vectors.
Execution completed
Read completed
Number of rows: 88
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (9928, 2)
Author:    27  Size:  2638
Author:    80  Size:  6423
Author:    81  Size:   867
Min: 867
Max: 6423
Authors [27, 80, 81].
Found 2601 texts.
Found 2601 labels.
Found 34712 unique tokens.
Shape of data tensor: (2601, 1000)
Shape of label tensor: (2601, 3)
Done compiling.
Train on 2080 samples, validate on 521 samples
Epoch 1/30
2080/2080 [==============================] - 193s - loss: 1.1095 - acc: 0.3317 - val_loss: 1.0933 - val_acc: 0.3685
Epoch 2/30
2080/2080 [==============================] - 193s - loss: 1.10

In [ ]:
%tb